In [1]:
import pandas as pd
import numpy as np
import csv
import difflib
from tqdm import tqdm
import nltk

tqdm.pandas(desc="Fuzzy Match Progress")

In [2]:
assignee_details = pd.read_csv(
    "./location_assignee/location_assignee.tsv", sep="\t", encoding="latin-1")
#assignee_details.location_id=assignee_details.location_id.astype(str)

location = pd.read_csv("./location/location.tsv", sep="\t", encoding="latin-1")
location.id = location.id.astype(str)
location = location[location.id != "nan"]
location = location.assign(country=location.country.str.lower())
location.country = location.country.astype(str)
location = location[location.country != "nan"]

us_locations = location[location.country == "us"]

us_assignees = pd.merge(
    left=assignee_details,
    right=us_locations,
    how="inner",
    left_on="location_id",
    right_on="id")

In [3]:
us_assignees_ids = us_assignees.assignee_id.unique()

In [10]:
assignee_details = pd.read_csv(
    "./assignee/assignee.tsv", sep="\t", low_memory=False, encoding='latin-1')

In [4]:
patent_assignee_mapping = pd.read_csv(
    "./patent_assignee/patent_assignee.tsv", sep="\t", low_memory=False)

patent_results = pd.read_csv(
    "../Phase-1-Search-Term/green-technology/green_utility_patents.csv",
    dtype={
        'Emerging low carbon-Additional energy sources':
        bool,
        'Emerging low carbon-All-purpose':
        bool,
        'Emerging low carbon-Alternative fuel':
        bool,
        'Emerging low carbon-Alternative fuel vehicle':
        bool,
        'Emerging low carbon-Battery':
        bool,
        'Emerging low carbon-Building technologies':
        bool,
        'Emerging low carbon-Carbon capture & storage':
        bool,
        'Emerging low carbon-Electrochemical processes':
        bool,
        'Emerging low carbon-Energy management':
        bool,
        'Environmental-Air pollution':
        bool,
        'Environmental-All-purpose':
        bool,
        'Environmental-Biological treatment':
        bool,
        'Environmental-Contaminated land reclamation & remediation':
        bool,
        'Environmental-Environmental monitoring, instrumentation and analysis':
        bool,
        'Environmental-Marine pollution control':
        bool,
        'Environmental-Noise & vibration control':
        bool,
        'Environmental-Recovery and recycling':
        bool,
        'Environmental-Waste management':
        bool,
        'Environmental-Water supply & waste water treatment':
        bool,
        'General-':
        bool,
        'Renewable energy-All-purpose':
        bool,
        'Renewable energy-Biomass':
        bool,
        'Renewable energy-Geothermal':
        bool,
        'Renewable energy-Photovoltaic & solar':
        bool,
        'Renewable energy-Wave & Tidal':
        bool,
        'Renewable energy-Wind':
        bool,
        'id':
        str,
        'synbio-Bio Tech/Engg':
        bool,
        'synbio-Cell biology':
        bool,
        'synbio-Chemical':
        bool,
        'synbio-General':
        bool,
        'synbio-Genetics':
        bool,
        'synbio-Nano technology':
        bool
    })

patent_results.id = patent_results.id.astype(str)
patent_results = patent_results[patent_results.id != "nan"]

patent_assignee_mapping.assignee_id = patent_assignee_mapping.assignee_id.astype(
    str)
patent_assignee_mapping.patent_id = patent_assignee_mapping.patent_id.astype(
    str)

patent_assignee_mapping = patent_assignee_mapping[
    patent_assignee_mapping.assignee_id != "nan"]
patent_assignee_mapping = patent_assignee_mapping[
    patent_assignee_mapping.patent_id != "nan"]

In [9]:
green_selected_patents = patent_results[patent_results.iloc[:, :26].any(
    axis=1)]
green_selected_patent_details = pd.merge(
    green_selected_patents,
    patent_assignee_mapping,
    how="inner",
    left_on="id",
    right_on="patent_id")

green_selected_patent_details.reset_index(drop=True, inplace=True)
green_selected_patent_details.assignee_id.fillna("", inplace=True)

us_green_selected_patent_details = green_selected_patent_details[
    green_selected_patent_details.assignee_id.isin(us_assignees_ids)]
us_green_selected_patent_details.shape

(1576, 9)

In [11]:
us_green_selected_patent_details.iloc[:, :6].apply(
    pd.value_counts).to_csv("latest-green-patent-selection-by-category.csv")

green_selected_patent_assignee_details = pd.merge(
    us_green_selected_patent_details,
    assignee_details,
    how="inner",
    left_on="assignee_id",
    right_on="id")

green_organizations = green_selected_patent_assignee_details
green_organizations = green_organizations.assign(
    compare_organization=green_organizations.organization.str.lower())
green_organizations.compare_organization = green_organizations.compare_organization.astype(
    str)
green_organizations = green_organizations[
    green_organizations.compare_organization != "nan"]

In [14]:
sam_entities = pd.read_csv("./sam-entities.csv", encoding="latin-1")

sam_entities = sam_entities.assign(
    compare_lbn=sam_entities.LEGAL_BUSINESS_NAME.str.lower())
sam_entities = sam_entities.assign(
    compare_dban=sam_entities.DBA_NAME.str.lower())
sam_entities = sam_entities.assign(
    COUNTRY_OF_INC=sam_entities.COUNTRY_OF_INC.str.lower())
#sam_entities.COUNTRY_OF_INC=sam_entities.COUNTRY_OF_INC.astype(str)
sam_entities.compare_dban = sam_entities.compare_dban.astype(str)
sam_entities.compare_lbn = sam_entities.compare_lbn.astype(str)
sam_entities = sam_entities[(sam_entities.compare_dban != "nan")
                            | (sam_entities.compare_lbn != "nan")]
sam_small_businesses = sam_entities[(
    ~pd.isnull(sam_entities.NAICS_CODE_COUNTER > 0) &
    sam_entities.NAICS_CODES.str.contains("Y") &
    (sam_entities.COUNTRY_OF_INC == "usa"))]

C:\Users\smadhavan\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,1,3,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
green_sam = pd.merge(
    left=green_organizations,
    right=sam_entities,
    how="inner",
    left_on="compare_organization",
    right_on="compare_lbn")
dba_sam_entities = sam_entities[~sam_entities.compare_dban.isnull()]
green_sam_by_dba = pd.merge(
    left=green_organizations,
    right=dba_sam_entities,
    how="inner",
    left_on="compare_organization",
    right_on="compare_dban")
complete_green_sam = pd.concat([green_sam, green_sam_by_dba], axis=0)
complete_green_sam.drop_duplicates(inplace=True)
len(complete_green_sam.compare_organization.unique())

154

In [41]:
complete_green_sam[["patent_id", "NAICS_CODES"]].to_csv(
    "green-sam-patents.csv", index=False)

In [35]:
non_sam_green_organizations = green_organizations[(
    (~green_organizations.compare_organization.isin(
        sam_entities.compare_lbn.tolist())) &
    (~green_organizations.compare_organization.isin(
        sam_entities.compare_dban.tolist())))]

In [ ]:
non_sam_green_organizations[["patent_id"]].to_csv(
    "green-non-sam-patents.csv", index=False)

In [37]:
small_business_green_sam = complete_green_sam[
    (complete_green_sam.NAICS_CODE_COUNTER > 0)
    & (complete_green_sam.NAICS_CODES.str.contains("Y"))]
len(small_business_green_sam.compare_organization.unique())

41

In [38]:
small_business_green_sam.head()

,Renewable energy-All-purpose,Renewable energy-Biomass,Renewable energy-Geothermal,Renewable energy-Photovoltaic & solar,Renewable energy-Wave & Tidal,Renewable energy-Wind,id_x,patent_id,assignee_id,id_y,...,BUSINESS_TYPES,PRIMARY_NAICS,NAICS_CODE_COUNTER,NAICS_CODES,PSC_CODE_COUNTER,PSC_CODES,SBA_BUSINESS_TYPES_COUNTER,SBA_BUSINESS_TYPES,compare_lbn,compare_dban
42,False,True,False,False,False,False,9200236,9200236,28fcb5a9ceedd00020295fa27575596b,28fcb5a9ceedd00020295fa27575596b,...,27~2X~LJ~VW,311119.0,3.0,311119Y~311225Y~325414Y,4.0,AA92~AD53~AE32~AG93,0.0,NaN,"heliae development, llc",nan
43,False,True,False,False,False,False,9416336,9416336,28fcb5a9ceedd00020295fa27575596b,28fcb5a9ceedd00020295fa27575596b,...,27~2X~LJ~VW,311119.0,3.0,311119Y~311225Y~325414Y,4.0,AA92~AD53~AE32~AG93,0.0,NaN,"heliae development, llc",nan
44,False,True,False,False,False,False,9200337,9200337,872772ef9abeaa7fb1448dbe6b6070f8,872772ef9abeaa7fb1448dbe6b6070f8,...,2X~MF,541711.0,1.0,541711Y,0.0,NaN,0.0,NaN,"renmatix, inc.",nan
45,False,True,False,False,False,False,9359651,9359651,872772ef9abeaa7fb1448dbe6b6070f8,872772ef9abeaa7fb1448dbe6b6070f8,...,2X~MF,541711.0,1.0,541711Y,0.0,NaN,0.0,NaN,"renmatix, inc.",nan
1151,True,False,False,True,False,False,9209648,9209648,696b1d873bbeaa9a773445ba4fabe56b,696b1d873bbeaa9a773445ba4fabe56b,...,2X,221114.0,23.0,221111Y~221114Y~237130Y~238120Y~238290Y~238910...,5.0,B543~C211~N054~N061~W061,0.0,NaN,"envision solar international, inc.",nan


In [20]:
small_business_missing_url = small_business_green_sam[pd.isnull(
    small_business_green_sam.CORPORATE_URL)]

small_business_missing_url.to_csv(
    "small_green_orgs_missing_url_latest.csv", index=False)

In [21]:
small_business_green_sam = small_business_green_sam.assign(
    lower_url=small_business_green_sam.CORPORATE_URL.str.lower())

In [22]:
gp = small_business_green_sam.groupby(["compare_organization", "lower_url"])

In [ ]:
us_green_orgs_missing_url

In [23]:
corp_urls = pd.DataFrame(gp.size())
corp_urls.shape

(27, 1)

In [26]:
corp_urls.to_csv("./green_business_urls.csv")

In [45]:
patent_assignee_mapping = pd.read_csv(
    "./patent_assignee/patent_assignee.tsv", sep="\t", low_memory=False)

patent_results = pd.read_csv(
    "../Phase-1-Search-Term/synthetic-biology/synbio_utility_patents.csv",
    dtype={
        'Emerging low carbon-Additional energy sources':
        bool,
        'Emerging low carbon-All-purpose':
        bool,
        'Emerging low carbon-Alternative fuel':
        bool,
        'Emerging low carbon-Alternative fuel vehicle':
        bool,
        'Emerging low carbon-Battery':
        bool,
        'Emerging low carbon-Building technologies':
        bool,
        'Emerging low carbon-Carbon capture & storage':
        bool,
        'Emerging low carbon-Electrochemical processes':
        bool,
        'Emerging low carbon-Energy management':
        bool,
        'Environmental-Air pollution':
        bool,
        'Environmental-All-purpose':
        bool,
        'Environmental-Biological treatment':
        bool,
        'Environmental-Contaminated land reclamation & remediation':
        bool,
        'Environmental-Environmental monitoring, instrumentation and analysis':
        bool,
        'Environmental-Marine pollution control':
        bool,
        'Environmental-Noise & vibration control':
        bool,
        'Environmental-Recovery and recycling':
        bool,
        'Environmental-Waste management':
        bool,
        'Environmental-Water supply & waste water treatment':
        bool,
        'General-':
        bool,
        'Renewable energy-All-purpose':
        bool,
        'Renewable energy-Biomass':
        bool,
        'Renewable energy-Geothermal':
        bool,
        'Renewable energy-Photovoltaic & solar':
        bool,
        'Renewable energy-Wave & Tidal':
        bool,
        'Renewable energy-Wind':
        bool,
        'id':
        str,
        'synbio-Bio Tech/Engg':
        bool,
        'synbio-Cell biology':
        bool,
        'synbio-Chemical':
        bool,
        'synbio-General':
        bool,
        'synbio-Genetics':
        bool,
        'synbio-Nano technology':
        bool
    })

patent_results.id = patent_results.id.astype(str)
patent_results = patent_results[patent_results.id != "nan"]

patent_assignee_mapping.assignee_id = patent_assignee_mapping.assignee_id.astype(
    str)
patent_assignee_mapping.patent_id = patent_assignee_mapping.patent_id.astype(
    str)

patent_assignee_mapping = patent_assignee_mapping[
    patent_assignee_mapping.assignee_id != "nan"]
patent_assignee_mapping = patent_assignee_mapping[
    patent_assignee_mapping.patent_id != "nan"]

In [46]:
patent_results.shape

(1694, 33)

In [47]:
synbio_selected_patents = patent_results[patent_results.iloc[:, 27:33].any(
    axis=1)]
synbio_selected_patent_details = pd.merge(
    synbio_selected_patents,
    patent_assignee_mapping,
    how="inner",
    left_on="id",
    right_on="patent_id")
synbio_selected_patent_details.reset_index(drop=True, inplace=True)
synbio_selected_patent_details.assignee_id.fillna("", inplace=True)

print(synbio_selected_patent_details.shape)

us_synbio_selected_patent_details = synbio_selected_patent_details[
    synbio_selected_patent_details.assignee_id.isin(us_assignees_ids)]
print(us_synbio_selected_patent_details.shape)

us_synbio_selected_patent_details.iloc[:, 27:33].apply(
    pd.value_counts).to_csv("latest-synbio-patent-selection-by-category.csv")
synbio_selected_patent_assignee_details = pd.merge(
    us_synbio_selected_patent_details,
    assignee_details,
    how="inner",
    left_on="assignee_id",
    right_on="id")

synbio_selected_patent_assignee_details.shape

(1806, 35)
(1277, 35)


(1277, 40)

In [48]:
len(synbio_selected_patent_assignee_details.organization.unique())

573

In [50]:
synbio_organizations = synbio_selected_patent_assignee_details
synbio_organizations = synbio_organizations.assign(
    compare_organization=synbio_organizations.organization.str.lower())
synbio_organizations.compare_organization = synbio_organizations.compare_organization.astype(
    str)
synbio_organizations = synbio_organizations[
    synbio_organizations.compare_organization != "nan"]
synbio_organizations.shape

(1277, 41)

In [51]:
synbio_sam = pd.merge(
    left=synbio_organizations,
    right=sam_entities,
    how="inner",
    left_on="compare_organization",
    right_on="compare_lbn")
dba_sam_entities = sam_entities[~sam_entities.compare_dban.isnull()]
synbio_sam_by_dba = pd.merge(
    left=synbio_organizations,
    right=dba_sam_entities,
    how="inner",
    left_on="compare_organization",
    right_on="compare_dban")
complete_synbio_sam = pd.concat([synbio_sam, synbio_sam_by_dba], axis=0)
complete_synbio_sam.drop_duplicates(inplace=True)
len(complete_synbio_sam.compare_organization.unique())

196

In [35]:
small_business_synbio_sam = complete_synbio_sam[
    (complete_synbio_sam.NAICS_CODE_COUNTER > 0)
    & (complete_synbio_sam.NAICS_CODES.str.contains("Y"))]
len(small_business_synbio_sam.compare_organization.unique())

39

In [52]:
complete_synbio_sam[["patent_id", "NAICS_CODES"]].to_csv(
    "synbio-sam-patents.csv", index=False)

In [53]:
non_sam_synbio_organizations = synbio_organizations[(
    (~synbio_organizations.compare_organization.isin(
        sam_entities.compare_lbn.tolist())) &
    (~synbio_organizations.compare_organization.isin(
        sam_entities.compare_dban.tolist())))]

In [56]:
non_sam_synbio_organizations[["patent_id"]].to_csv(
    "synbio-non-sam-patents.csv", index=False)

In [36]:
small_business_missing_url = small_business_synbio_sam[pd.isnull(
    small_business_synbio_sam.CORPORATE_URL)]

small_business_missing_url.to_csv(
    "small_synbio_orgs_missing_url_latest.csv", index=False)

In [37]:
small_business_synbio_sam = small_business_synbio_sam.assign(
    lower_url=small_business_synbio_sam.CORPORATE_URL.str.lower())

In [38]:
gp = small_business_synbio_sam.groupby(["compare_organization", "lower_url"])
corp_urls = pd.DataFrame(gp.size())
corp_urls.to_csv("./synbio_business_urls.csv")

In [ ]:
us_green_orgs_missing_url = small_business_green_sam[pd.isnull(
    small_business_green_sam.lower_url)]

In [ ]:
green_url_missing_orgs = pd.read_csv(
    "./small_green_patents_orgs_missing_url.csv")

In [76]:
org_list = us_green_orgs_missing_url[[
    "LEGAL_BUSINESS_NAME", "DBA_NAME", "PHYSICAL_ADDRESS_LINE_1",
    "PHYSICAL_ADDRESS_LINE_2", "PHYSICAL_ADDRESS_CITY",
    "PHYSICAL_ADDRESS_PROVINCE_OR_STATE", "PHYSICAL_ADDRESS_ZIP",
    "PHYSICAL_ADDRESS_ZIP4", "PHYSICAL_ADDRESS_COUNTRY_CODE",
    "ENTITY_CONGRESSIONAL_DISTRICT", "CORPORATE_URL", "ENTITY_STRUCTURE",
    "STATE_OF INC", "COUNTRY_OF_INC"
]]

In [77]:
org_list.drop_duplicates(inplace=True)

C:\Users\smadhavan\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
org_list.head()

,LEGAL_BUSINESS_NAME,DBA_NAME,PHYSICAL_ADDRESS_LINE_1,PHYSICAL_ADDRESS_LINE_2,PHYSICAL_ADDRESS_CITY,PHYSICAL_ADDRESS_PROVINCE_OR_STATE,PHYSICAL_ADDRESS_ZIP,PHYSICAL_ADDRESS_ZIP4,PHYSICAL_ADDRESS_COUNTRY_CODE,ENTITY_CONGRESSIONAL_DISTRICT,CORPORATE_URL,ENTITY_STRUCTURE,STATE_OF INC,COUNTRY_OF_INC
137986,SMITH FILTER CORPORATION,NaN,5000 41ST ST CT,NaN,MOLINE,IL,61265,7583.0,USA,17.0,NaN,2L,IL,usa
137987,URBAN RESEARCH & DEVELOPMENT CORPORATION,NaN,28 W BROAD ST,NaN,BETHLEHEM,PA,18018,5705.0,USA,15.0,NaN,2L,PA,usa
139002,"BOSTON MEDICAL PRODUCTS, INC.",NaN,70 CHESTNUT ST,NaN,SHREWSBURY,MA,1545,4101.0,USA,2.0,NaN,2L,MA,usa
140481,THE EASTERN COMPANY,EBERHARD MFG DIV,21944 DRAKE RD,NaN,STRONGSVILLE,OH,44149,6609.0,USA,16.0,NaN,2L,CT,usa
140812,MICRON CORPORATION,MICRON,1940 ELK RIVER DAM RD,NaN,TULLAHOMA,TN,37388,9716.0,USA,4.0,NaN,2L,TN,usa
